In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

In [2]:
# Imports
import sys
import numpy as np
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3

In [3]:
np.random.seed(20)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
%matplotlib inline

# Data

In [5]:
# Read data
motion_01 = np.genfromtxt('./motion_01.csv', delimiter=',')
motion_02 = np.genfromtxt('./motion_02.csv', delimiter=',')
motion_03 = np.genfromtxt('./motion_03.csv', delimiter=',')
motion_04 = np.genfromtxt('./motion_04.csv', delimiter=',')

print("Motion 0.1 data has a length of ", len(motion_01))
print("Motion 0.2 data has a length of ", len(motion_02))
print("Motion 0.3 data has a length of ", len(motion_03))
print("Motion 0.4 data has a length of ", len(motion_04))

Motion 0.1 data has a length of  105172
Motion 0.2 data has a length of  70114
Motion 0.3 data has a length of  53550
Motion 0.4 data has a length of  72082


In [6]:
# Stack all the data together
D = np.concatenate((motion_01, motion_02, motion_03, motion_04))
print("The full data matrix D has a shape of: ", D.shape)

The full data matrix D has a shape of:  (300918, 67)


## Analyze Command Inputs

The command input consists of the following inputs:

- tick **(not considered)**
- mode **(not considered)**
- yaw
- roll
- pitch
- sideSpeed
- rotateSpeed
- forwardSpeed
- bodyHeight
- footRaiseHeight

In [7]:
# Values in the dataset w.r.t to robot rotation
print("Yaw values: ", set(D[:, 7]))
print("Roll values: ", set(D[:, 9]))
print("Pitch values: ", set(D[:, 8]))

Yaw values:  {0.0}
Roll values:  {0.0}
Pitch values:  {0.0}


In [8]:
# Values in the dataset w.r.t to robot velocities
print("sideSpeed values: ", set(D[:, 3]))
print("rotateSpeed values: ", set(D[:, 4]))
print("forwardSpeed values: ", set(D[:, 2]))

sideSpeed values:  {0.0, 0.1, -0.214359, 0.214359, -0.1, 0.313843, -0.313843, -0.417725, 0.417725}
rotateSpeed values:  {0.0, 0.1, 0.214359, -0.214359, -0.1, 0.313843, -0.313843, -0.417725, 0.417725}
forwardSpeed values:  {0.0, 0.1, -0.214359, 0.214359, -0.1, 0.313843, -0.313843, 0.417725, -0.417725}


In [9]:
# Values in the dataset w.r.t to robot height
print("bodyHeight values: ", set(D[:, 5]))
print("footRaiseHeight values: ", set(D[:, 6]))

bodyHeight values:  {0.0}
footRaiseHeight values:  {0.0}


## Analyze Initial Cmd/State Mappings

The considered robot **HighState** attributes are:

- commVersion **(not considered)**
- robotID **(not considered)**
- SN **(not considered)**
- bandWidth **(not considered)**
- mode **(not considered)**
- imu
- forwardSpeed
- sideSpeed
- rotateSpeed
- bodyHeight
- updownSpeed
- forwardPosition
- sidePosition
- footPosition2Body
- footSpeed2Body
- footForce
- footForceEst
- wirelessRemote **(not considered)**
- reserve **(not considered)**
- crc **(not considered)**

In [10]:
# Entries where command and state are similar (D[:, 2:5] IS THE REFERENCE VALUE)
matching_command_and_state_idx = np.where(np.all(np.isclose(D[:, 25:28], D[:, 2:5], rtol=0, atol=0.05), axis=1))[0]
len(matching_command_and_state_idx)

59837

In [11]:
for match in matching_command_and_state_idx:
    print(D[match, 2:5], D[match, 25:28])
    break

[0. 0. 0.] [-0.00028487 -0.00046512  0.00576802]


### State velocities

### Standing mode velocities

In [1]:
#D[matching_command_and_state_idx, 2:5]

In [23]:
# All entries with zero velocity command
no_motion_commands_idx = np.where(~np.any(D[:, 2:5], axis=1))[0]
no_motion_commands_idx = np.intersect1d(matching_command_and_state_idx, no_motion_commands_idx)
len(no_motion_commands_idx)

12845

In [24]:
# All entries with velocity command
motion_commands_idx = np.where(np.any(D[:, 2:5], axis=1))[0]
motion_commands_idx = np.intersect1d(matching_command_and_state_idx, motion_commands_idx)
len(motion_commands_idx)

15791

In [2]:
#D[no_motion_commands_idx, 2:5]

In [3]:
#D[motion_commands_idx, 2:5]

In [27]:
# Forward speed values for standing mode
print("Min forward speed: ", min(D[no_motion_commands_idx, 25]))
print("Max forward speed: ", max(D[no_motion_commands_idx, 25]))
print("Avg forward speed: ", np.mean(D[no_motion_commands_idx, 25]))

Min forward speed:  -0.0496093
Max forward speed:  0.0499541
Avg forward speed:  0.001889097105845076


In [28]:
# Forward speed values for walking mode
print("Min forward speed: ", min(D[motion_commands_idx, 25]))
print("Max forward speed: ", max(D[motion_commands_idx, 25]))
print("Avg forward speed: ", np.mean(D[motion_commands_idx, 25]))

Min forward speed:  -0.148984
Max forward speed:  0.149998
Avg forward speed:  0.06692352462522828


In [29]:
# Side speed values for standing mode
print("Min side speed: ", min(D[no_motion_commands_idx, 26]))
print("Max side speed: ", max(D[no_motion_commands_idx, 26]))
print("Avg side speed: ", np.mean(D[no_motion_commands_idx, 26]))

Min side speed:  -0.0490406
Max side speed:  0.0499679
Avg side speed:  0.002147428477626205


In [30]:
# Side speed values for walking mode
print("Min side speed: ", min(D[motion_commands_idx, 26]))
print("Max side speed: ", max(D[motion_commands_idx, 26]))
print("Avg side speed: ", np.mean(D[motion_commands_idx, 26]))

Min side speed:  -0.107743
Max side speed:  0.124038
Avg side speed:  0.005747958247531505


In [31]:
# Rotate speed values for standing mode
print("Min rotate speed: ", min(D[no_motion_commands_idx, 27]))
print("Max rotate speed: ", max(D[no_motion_commands_idx, 27]))
print("Avg rotate speed: ", np.mean(D[no_motion_commands_idx, 27]))

Min rotate speed:  -0.0496622
Max rotate speed:  0.0494074
Avg rotate speed:  -0.001076552834799533


In [32]:
# Rotate speed values for walking mode
print("Min rotate speed: ", min(D[motion_commands_idx, 27]))
print("Max rotate speed: ", max(D[motion_commands_idx, 27]))
print("Avg rotate speed: ", np.mean(D[motion_commands_idx, 27]))

Min rotate speed:  -0.149797
Max rotate speed:  0.149542
Avg rotate speed:  0.0018434844357545458


### Standing mode footForce

In [33]:
# Front right foot force (standing)
print("Min force (FR): ", min(D[no_motion_commands_idx, 56]))
print("Max force (FR): ", max(D[no_motion_commands_idx, 56]))

Min force (FR):  -5.0
Max force (FR):  164.0


In [34]:
# Front right foot force (walking)
print("Min force (FR): ", min(D[motion_commands_idx, 56]))
print("Max force (FR): ", max(D[motion_commands_idx, 56]))

Min force (FR):  -8.0
Max force (FR):  175.0


In [35]:
# Front left foot force (standing)
print("Min force (FL): ", min(D[no_motion_commands_idx, 57]))
print("Max force (FL): ", max(D[no_motion_commands_idx, 57]))

Min force (FL):  -5.0
Max force (FL):  188.0


In [36]:
# Front left foot force (walking)
print("Min force (FL): ", min(D[motion_commands_idx, 57]))
print("Max force (FL): ", max(D[motion_commands_idx, 57]))

Min force (FL):  -5.0
Max force (FL):  207.0


In [37]:
# Right rear (standing)
print("Min force (RR): ", min(D[no_motion_commands_idx, 58]))
print("Max force (RR): ", max(D[no_motion_commands_idx, 58]))

Min force (RR):  -4.0
Max force (RR):  129.0


In [38]:
# Right rear (walking)
print("Min force (RR): ", min(D[motion_commands_idx, 58]))
print("Max force (RR): ", max(D[motion_commands_idx, 58]))

Min force (RR):  -6.0
Max force (RR):  139.0


In [39]:
# Left rear (standing)
print("Min force (LR): ", min(D[no_motion_commands_idx, 59]))
print("Max force (LR): ", max(D[no_motion_commands_idx, 59]))

Min force (LR):  -4.0
Max force (LR):  149.0


In [40]:
# Left rear (walking)
print("Min force (LR): ", min(D[motion_commands_idx, 59]))
print("Max force (LR): ", max(D[motion_commands_idx, 59]))

Min force (LR):  -6.0
Max force (LR):  158.0


### Standing mode footSpeed2Body

In [41]:
# Front right foot velocity
print("Min_x foot velocity (FR): ", min(D[no_motion_commands_idx, 44]))
print("Max_x foot velocity (FR): ", max(D[no_motion_commands_idx, 44]))
print("Min_y foot velocity (FR): ", min(D[no_motion_commands_idx, 45]))
print("Max_y foot velocity (FR): ", max(D[no_motion_commands_idx, 45]))
print("Min_z foot velocity (FR): ", min(D[no_motion_commands_idx, 46]))
print("Max_z foot velocity (FR): ", max(D[no_motion_commands_idx, 46]))

Min_x foot velocity (FR):  -1.09183
Max_x foot velocity (FR):  0.860323
Min_y foot velocity (FR):  -0.308034
Max_y foot velocity (FR):  0.480257
Min_z foot velocity (FR):  -1.84169
Max_z foot velocity (FR):  2.24745


In [42]:
# Front left foot velocity
print("Min_x foot velocity (FL): ", min(D[no_motion_commands_idx, 47]))
print("Max_x foot velocity (FL): ", max(D[no_motion_commands_idx, 47]))
print("Min_y foot velocity (FL): ", min(D[no_motion_commands_idx, 48]))
print("Max_y foot velocity (FL): ", max(D[no_motion_commands_idx, 48]))
print("Min_z foot velocity (FL): ", min(D[no_motion_commands_idx, 49]))
print("Max_z foot velocity (FL): ", max(D[no_motion_commands_idx, 49]))

Min_x foot velocity (FL):  -1.01301
Max_x foot velocity (FL):  0.663398
Min_y foot velocity (FL):  -0.551442
Max_y foot velocity (FL):  0.321819
Min_z foot velocity (FL):  -1.80567
Max_z foot velocity (FL):  1.78729


In [43]:
# Rear right foot velocity
print("Min_x foot velocity (RR): ", min(D[no_motion_commands_idx, 50]))
print("Max_x foot velocity (RR): ", max(D[no_motion_commands_idx, 50]))
print("Min_y foot velocity (RR): ", min(D[no_motion_commands_idx, 51]))
print("Max_y foot velocity (RR): ", max(D[no_motion_commands_idx, 51]))
print("Min_z foot velocity (RR): ", min(D[no_motion_commands_idx, 52]))
print("Max_z foot velocity (RR): ", max(D[no_motion_commands_idx, 52]))

Min_x foot velocity (RR):  -1.06615
Max_x foot velocity (RR):  0.56607
Min_y foot velocity (RR):  -0.445356
Max_y foot velocity (RR):  0.452863
Min_z foot velocity (RR):  -1.75605
Max_z foot velocity (RR):  1.57756


In [44]:
# Rear left foot velocity
print("Min_x foot velocity (LR): ", min(D[no_motion_commands_idx, 53]))
print("Max_x foot velocity (LR): ", max(D[no_motion_commands_idx, 53]))
print("Min_y foot velocity (LR): ", min(D[no_motion_commands_idx, 54]))
print("Max_y foot velocity (LR): ", max(D[no_motion_commands_idx, 54]))
print("Min_z foot velocity (LR): ", min(D[no_motion_commands_idx, 55]))
print("Max_z foot velocity (LR): ", max(D[no_motion_commands_idx, 55]))

Min_x foot velocity (LR):  -1.074
Max_x foot velocity (LR):  0.910961
Min_y foot velocity (LR):  -0.676132
Max_y foot velocity (LR):  0.347886
Min_z foot velocity (LR):  -1.89147
Max_z foot velocity (LR):  2.18542


### Standing mode footPosition2Body

In [45]:
# Front right foot velocity
print("Min_x foot position (FR): ", min(D[no_motion_commands_idx, 32]))
print("Max_x foot position (FR): ", max(D[no_motion_commands_idx, 32]))
print("Min_y foot position (FR): ", min(D[no_motion_commands_idx, 33]))
print("Max_y foot position (FR): ", max(D[no_motion_commands_idx, 33]))
print("Min_z foot position (FR): ", min(D[no_motion_commands_idx, 34]))
print("Max_z foot position (FR): ", max(D[no_motion_commands_idx, 34]))

Min_x foot position (FR):  0.221526
Max_x foot position (FR):  0.269551
Min_y foot position (FR):  -0.148527
Max_y foot position (FR):  -0.118207
Min_z foot position (FR):  -0.418963
Max_z foot position (FR):  -0.280531


In [46]:
# Front left foot velocity
print("Min_x foot position (FL): ", min(D[no_motion_commands_idx, 35]))
print("Max_x foot position (FL): ", max(D[no_motion_commands_idx, 35]))
print("Min_y foot position (FL): ", min(D[no_motion_commands_idx, 36]))
print("Max_y foot position (FL): ", max(D[no_motion_commands_idx, 36]))
print("Min_z foot position (FL): ", min(D[no_motion_commands_idx, 37]))
print("Max_z foot position (FL): ", max(D[no_motion_commands_idx, 37]))

Min_x foot position (FL):  0.217673
Max_x foot position (FL):  0.264232
Min_y foot position (FL):  0.122577
Max_y foot position (FL):  0.151418
Min_z foot position (FL):  -0.413366
Max_z foot position (FL):  -0.279251


In [47]:
# Rear right foot position
print("Min_x foot position (RR): ", min(D[no_motion_commands_idx, 38]))
print("Max_x foot position (RR): ", max(D[no_motion_commands_idx, 38]))
print("Min_y foot position (RR): ", min(D[no_motion_commands_idx, 39]))
print("Max_y foot position (RR): ", max(D[no_motion_commands_idx, 39]))
print("Min_z foot position (RR): ", min(D[no_motion_commands_idx, 40]))
print("Max_z foot position (RR): ", max(D[no_motion_commands_idx, 40]))

Min_x foot position (RR):  -0.257951
Max_x foot position (RR):  -0.216334
Min_y foot position (RR):  -0.148205
Max_y foot position (RR):  -0.119507
Min_z foot position (RR):  -0.421237
Max_z foot position (RR):  -0.292146


In [48]:
# Rear left foot position
print("Min_x foot position (RL): ", min(D[no_motion_commands_idx, 41]))
print("Max_x foot position (RL): ", max(D[no_motion_commands_idx, 41]))
print("Min_y foot position (RL): ", min(D[no_motion_commands_idx, 42]))
print("Max_y foot position (RL): ", max(D[no_motion_commands_idx, 42]))
print("Min_z foot position (RL): ", min(D[no_motion_commands_idx, 43]))
print("Max_z foot position (RL): ", max(D[no_motion_commands_idx, 43]))

Min_x foot position (RL):  -0.255788
Max_x foot position (RL):  -0.207412
Min_y foot position (RL):  0.124558
Max_y foot position (RL):  0.155336
Min_z foot position (RL):  -0.423099
Max_z foot position (RL):  -0.284516
